In [32]:
import dolfin as dl

In [59]:
num_mesh_points = 10
mesh_bounds = [0, 1]
num_components_variables = 2
element_type_variables = "Lagrange"
element_type_parameters = "Lagrange"
element_degree_variables = 1
element_degree_parameters = 1

In [60]:
mesh = dl.IntervalMesh(num_mesh_points, *mesh_bounds)
domain_dim = mesh.geometry().dim()

In [61]:
elem_variables = dl.VectorElement(
    family=element_type_variables,
    cell=mesh.ufl_cell(),
    degree=element_degree_variables,
    dim=num_components_variables,
)
elem_drift = dl.VectorElement(
    family=element_type_parameters,
    cell=mesh.ufl_cell(),
    degree=element_degree_parameters,
    dim=domain_dim,
)
elem_diffusion = dl.TensorElement(
    family=element_type_parameters,
    cell=mesh.ufl_cell(),
    degree=element_degree_parameters,
    shape=(domain_dim, domain_dim),
    symmetry=True,
)
elem_composite = dl.MixedElement([elem_drift, elem_diffusion])

func_space_variables = dl.FunctionSpace(mesh, elem_variables)
func_space_drift = dl.FunctionSpace(mesh, elem_drift)
func_space_diffusion = dl.FunctionSpace(mesh, elem_diffusion)
func_space_composite = dl.FunctionSpace(mesh, elem_composite)

In [63]:
def weak_form_mean_exit_time(
    forward_variable: dl.Function,
    adjoint_variable: dl.Function,
    drift: dl.Function,
    squared_diffusion: dl.Function,
) -> dl.Form:
    weak_form = (
        dl.dot(drift * adjoint_variable, dl.grad(forward_variable)) * dl.dx
        - 0.5
        * dl.dot(dl.div(squared_diffusion * adjoint_variable), dl.grad(forward_variable))
        * dl.dx
        + dl.Constant(1) * adjoint_variable * dl.dx
    )
    return weak_form


def weak_form_mean_exit_time_moments(
    forward_variable: dl.Function,
    adjoint_variable: dl.Function,
    drift: dl.Function,
    squared_diffusion: dl.Function,
) -> dl.Form:
    weak_form_component_1 = (
        dl.dot(drift * adjoint_variable[0], dl.grad(forward_variable[0])) * dl.dx
        - 0.5
        * dl.dot(dl.div(squared_diffusion * adjoint_variable[0]), dl.grad(forward_variable[0]))
        * dl.dx
        + dl.Constant(1) * adjoint_variable[0] * dl.dx
    )
    weak_form_component_2 = (
        dl.dot(drift * adjoint_variable[1], dl.grad(forward_variable[1])) * dl.dx
        - 0.5
        * dl.dot(dl.div(squared_diffusion * adjoint_variable[1]), dl.grad(forward_variable[1]))
        * dl.dx
        + 2 * forward_variable[0] * adjoint_variable[1] * dl.dx
    )
    weak_form = weak_form_component_1 + weak_form_component_2
    return weak_form


def weak_form_fokker_planck(
    forward_variable: dl.Function,
    adjoint_variable: dl.Function,
    drift: dl.Function,
    squared_diffusion: dl.Function,
) -> dl.Form:
    weak_form = (
        dl.div(drift * forward_variable) * adjoint_variable * dl.dx
        + 0.5
        * dl.dot(dl.div(squared_diffusion * forward_variable), dl.grad(adjoint_variable))
        * dl.dx
        + dl.Constant(0) * adjoint_variable * dl.dx
    )
    return weak_form

In [76]:
boundary_condition = dl.DirichletBC(
    func_space_variables, dl.Constant((0.0, 0.0)), lambda _, on_boundary: on_boundary
)